In [19]:
import spacy
from spacy.tokens import Token
from spacy import displacy
import pandas as pd
import pickle
import random
import tqdm

In [13]:
all_verbs = []

In [17]:
file = open("../stored_data/docs.obj",'rb')
docs = pickle.load(file)
file.close()

In [4]:
id = random.sample(list(docs.keys()), 1)
random_doc = docs[id[0]]
full_doc = ""
for sent in random_doc:
    full_doc += sent

In [5]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_lg")
entities = nlp.get_pipe('ner').labels



In [6]:
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [
    {"ENT_TYPE": {'IN': entities}}, 
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {'POS': 'VERB'},
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {"ENT_TYPE": {'IN': entities}}]
matcher.add("any_verb", [pattern])

In [7]:
doc = nlp("Fairchild Corp was acquired by Fujitsu.")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

13200282023714757851 any_verb 1 6 Corp was acquired by Fujitsu


In [28]:
all_verbs = []
for id, file in tqdm.tqdm(docs.items()):
    full_doc = ""
    for sent in file:
        full_doc += sent

    doc = nlp(full_doc)

    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span

    for token in span:
        if token.pos_ == "VERB":
            all_verbs.append(token)

100%|██████████| 8399/8399 [03:42<00:00, 37.74it/s]


In [ ]:
print(set(all_verbs))

In [35]:
similarity_matrix = [[] for x in range(len(set(all_verbs)))]

for idx, token1 in enumerate(set(all_verbs)):
    print(idx/len(set(all_verbs)), end="\r")
    for token2 in set(all_verbs):
        similarity_matrix[idx].append(token1.similarity(token2))


C:\Users\aubin\AppData\Local\Temp\ipykernel_16532\3095854996.py:6: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_matrix[idx].append(token1.similarity(token2))


KeyboardInterrupt: 

In [36]:
all_verbs_txt = [verb.text for verb in all_verbs]

In [37]:
filehandler = open("../stored_data/all_verbs.obj","wb")
pickle.dump(all_verbs_txt, filehandler)
filehandler.close()